In [3]:
import pandas as pd

data = pd.read_csv("measures_v2.csv")   # if your file name different, tell me

data.head()


,u_q,coolant,stator_winding,u_d,stator_tooth,motor_speed,i_d,i_q,pm,stator_yoke,ambient,torque,profile_id
0,-0.450682,18.805172,19.086670,-0.350055,18.293219,0.002866,0.004419,0.000328,24.554214,18.316547,19.850691,0.187101,17
1,-0.325737,18.818571,19.092390,-0.305803,18.294807,0.000257,0.000606,-0.000785,24.538078,18.314955,19.850672,0.245417,17
2,-0.440864,18.828770,19.089380,-0.372503,18.294094,0.002355,0.001290,0.000386,24.544693,18.326307,19.850657,0.176615,17
3,-0.327026,18.835567,19.083031,-0.316199,18.292542,0.006105,0.000026,0.002046,24.554018,18.330833,19.850647,0.238303,17
4,-0.471150,18.857033,19.082525,-0.332272,18.291428,0.003133,-0.064317,0.037184,24.565397,18.326662,19.850639,0.208197,17


In [4]:
data.columns

Index(['u_q', 'coolant', 'stator_winding', 'u_d', 'stator_tooth',
       'motor_speed', 'i_d', 'i_q', 'pm', 'stator_yoke', 'ambient', 'torque',
       'profile_id'],
      dtype='object')

In [5]:
data.head()

,u_q,coolant,stator_winding,u_d,stator_tooth,motor_speed,i_d,i_q,pm,stator_yoke,ambient,torque,profile_id
0,-0.450682,18.805172,19.086670,-0.350055,18.293219,0.002866,0.004419,0.000328,24.554214,18.316547,19.850691,0.187101,17
1,-0.325737,18.818571,19.092390,-0.305803,18.294807,0.000257,0.000606,-0.000785,24.538078,18.314955,19.850672,0.245417,17
2,-0.440864,18.828770,19.089380,-0.372503,18.294094,0.002355,0.001290,0.000386,24.544693,18.326307,19.850657,0.176615,17
3,-0.327026,18.835567,19.083031,-0.316199,18.292542,0.006105,0.000026,0.002046,24.554018,18.330833,19.850647,0.238303,17
4,-0.471150,18.857033,19.082525,-0.332272,18.291428,0.003133,-0.064317,0.037184,24.565397,18.326662,19.850639,0.208197,17


In [6]:
X = data.drop("pm", axis=1)
y = data["pm"]

print("Input shape:", X.shape)
print("Output shape:", y.shape)

Input shape: (1330816, 12)
Output shape: (1330816,)


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training data:", X_train.shape)
print("Testing data:", X_test.shape)

Training data: (1064652, 12)
Testing data: (266164, 12)


In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

lr = LinearRegression()
lr.fit(X_train, y_train)

pred_lr = lr.predict(X_test)

print("Linear Regression Results")
print("RMSE:", np.sqrt(mean_squared_error(y_test, pred_lr)))
print("R2 Score:", r2_score(y_test, pred_lr))


Linear Regression Results
RMSE: 7.227018368455066
R2 Score: 0.8554179684870349


In [12]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)

pred_dt = dt.predict(X_test)

print("Decision Tree Results")
print("RMSE:", np.sqrt(mean_squared_error(y_test, pred_dt)))
print("R2 Score:", r2_score(y_test, pred_dt))


Decision Tree Results
RMSE: 0.5162968419314334
R2 Score: 0.9992621043624889


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

pred_rf = rf.predict(X_test)

print("Random Forest Results")
print("RMSE:", np.sqrt(mean_squared_error(y_test, pred_rf)))
print("R2 Score:", r2_score(y_test, pred_rf))


In [ ]:
import pickle

pickle.dump(rf, open("model.pkl", "wb"))

print("Model Saved Successfully!")


In [5]:
from flask import Flask, request, jsonify, render_template
import numpy as np
import joblib
import traceback
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

try:
    manual_model = joblib.load("best_model.pkl")
    manual_scaler = joblib.load("scaler.pkl")
    print("✅ Manual model and scaler loaded.")
except Exception as e:
    print("❌ Error loading manual model or scaler:", e)

try:
    sensor_model = joblib.load("sensor_model.pkl")
    sensor_scaler = joblib.load("sensor_scaler.pkl")
    print("✅ Sensor model and scaler loaded.")
except Exception as e:
    print("❌ Error loading sensor model or scaler:", e)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/Manual')
def manual_page():
    return render_template('Manual.html')

@app.route('/sensor')
def sensor_page():
    return render_template('sensor.html')

# --------- Manual Prediction (JS Frontend API) ---------
@app.route("/predict", methods=["POST"])
def predict_manual():
    try:
        data = request.get_json()
        features = data["features"]
        features_array = np.array([features])
        scaled_features = manual_scaler.transform(features_array)
        prediction = manual_model.predict(scaled_features)[0]
        return jsonify({"prediction": round(prediction, 2)})
    except Exception as e:
        print("Manual prediction error:", e)
        print(traceback.format_exc())
        return jsonify({"error": str(e), "trace": traceback.format_exc()})

@app.route('/predict_sensor', methods=['POST'])
def predict_sensor():
    try:
        features = [
            float(request.form['ambient']),
            float(request.form['torque']),
            float(request.form['coolant']),
            float(request.form['u_d']),
            float(request.form['u_q']),
            float(request.form['motor_speed']),
            float(request.form['i_d']),
            float(request.form['i_q'])
        ]
        final_features = np.array(features).reshape(1, -1)
        scaled_features = sensor_scaler.transform(final_features)
        prediction = sensor_model.predict(scaled_features)
        output = round(prediction[0], 2)
        return render_template('sensor.html', prediction=f'Motor Temp: {output} °C')
    except Exception as e:
        return render_template('sensor.html', prediction=f'Error: {e}')
    
if __name__ == '__main__':
    app.run(debug=True)

ModuleNotFoundError: No module named 'flask_cors'

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import joblib

# Load data
data = pd.read_csv('measures_v2.csv')
data.dropna(inplace=True)

# Define features and target
features = ['ambient', 'torque', 'coolant', 'u_d', 'u_q', 'motor_speed', 'i_d', 'i_q']
X = data[features]
y = data['pm']  # Motor temperature

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=10, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Save model and scaler (🔁 both must match)
joblib.dump(model, 'sensor_model.pkl')
joblib.dump(scaler, 'sensor_scaler.pkl')

# Optional evaluation
score = model.score(X_test, y_test)
print(f"Model R^2 Score: {score:.2f}")
print("Sensor model + scaler trained and saved ✅")

Model R^2 Score: 0.69
Sensor model + scaler trained and saved ✅


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pickle

# Load dataset
data = pd.read_csv("measures_v2.csv")

# Select input features and target
X = data.drop("stator_winding", axis=1)
y = data["stator_winding"]

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler
pickle.dump(scaler, open("scaler.pkl", "wb"))

# Train models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(n_estimators=10, max_depth=5),
   # "SVM": SVR()
}

best_model = None
best_score = float('inf')

for name, model in models.items():
    try:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred)
        print(f"{name} MSE: {mse:.2f}")

        if mse < best_score:
            best_score = mse
            best_model = model
    except Exception as e:
        print(f"{name} failed: {e}")

# Save best model
if best_model:
    pickle.dump(best_model, open("best_model.pkl", "wb"))
    print("✅ Best model saved as best_model.pkl")
else:
    print("❌ No model trained successfully.")

Linear Regression MSE: 3.38
Decision Tree MSE: 0.16
Random Forest MSE: 19.00
✅ Best model saved as best_model.pkl
